In [1]:

# contains folders locations and all
from setup import *

from cells_detection.cell import *
from cells_detection.cellConfig import *
from cells_detection.detection_model import *
from cells_detection.train_model import *
from itertools import product
from mrcnn import model as modellib
from mrcnn import visualize
from math import ceil

TRAIN_SAVE_DIR = "train_results_all"

Root directory /users/nfs/Etu7/21113797/PRAT/mrcnn


In [2]:
from tensorboard import notebook
notebook.list()

Known TensorBoard instances:
  - port 8080: logdir train_results_all (started 1 day, 6:57:58 ago; pid 85127)


In [3]:
%load_ext tensorboard 
%tensorboard --logdir {TRAIN_SAVE_DIR} --host localhost --port 8080

Reusing TensorBoard on port 8080 (pid 85127), started 1 day, 6:57:59 ago. (Use '!kill 85127' to kill it.)

In [ ]:
params = []
# already done
# params+= [[20,20,20,"heads_4+_all",True,"SGD",1e-3,6,"coco"]]

# params+= [[20,20,20,"heads_4+_all",True,"ADAM",1e-4,6,"coco"]]


# params+= [[20,20,0,"heads_all",True,"SGD",1e-3,6,"coco"]]
# params+= [[20,20,0,"heads_all",True,"ADAM",1e-4,6,"coco"]]

# # it s a plus
# params+= [[10,20,20,"heads_4+_all",True,"SGD",1e-3,6,"coco"]]
# params+= [[10,20,20,"heads_4+_all",True,"ADAM",1e-4,6,"coco"]]

# NEW TESTS

params+= [[20,30,30,"heads_4+_all",True,"SGD",1e-3,6,"coco"]]

params+=[[20,30,0,"heads_all",True,"SGD",1e-3,6,"coco"]]

for (epochs1,epochs2,epochs3,train_mode,is_aug,optimizer,lr,bsize,init_weights) in params: 

    print("Weights: ", init_weights)
    print("Dataset: ", DATASET_DIR)

    # Configurations
    config = CellConfig(epochs1,is_aug,bsize,optimizer,lr)  

     # getting datasets with/out random augmenntations
    dataset_train, dataset_val, augmentation = prepareDatasets(DATASET_DIR,config)


    # selecting training mode


    modes = train_mode.split('_') # it always return a list even if there s no split careful maam

    if(len(modes)==1):
        
        config.TRAINABLE_LAYERS = modes[0]
        config.display()

        # Create model
        model = modellib.MaskRCNN(mode="training", config=config,
                                        model_dir=TRAIN_SAVE_DIR)
        # select the weights 
        model = selectWeights(model, weights= init_weights)
        # Train 
        train(model, config,dataset_train, dataset_val, augmentation)
        
# SECOND TRAINING PROCESS

    if(len(modes)==2): # aka it s equal to 2

        #### first train the head
        config.TRAINABLE_LAYERS = modes[0]        
        config.display()

        # Create model
        model = modellib.MaskRCNN(mode="training", config=config,
                                        model_dir=TRAIN_SAVE_DIR)
        # select the weights 
        model = selectWeights(model, weights= init_weights)
       
        train(model, config,dataset_train, dataset_val, augmentation)
        
        ### training everything
        config.TRAINABLE_LAYERS = modes[1]
        config.EPOCHS = epochs1 + epochs2
#         config.LEARNING_RATE /=10
        train(model, config,dataset_train, dataset_val, augmentation)

    
    if(len(modes)==3): # aka it s equal to 3

        #### first train the head
        config.TRAINABLE_LAYERS = modes[0]
        config.display()

        # Create model
        model = modellib.MaskRCNN(mode="training", config=config,
                                        model_dir=TRAIN_SAVE_DIR)
        # select the weights 
        model = selectWeights(model, weights= init_weights)
       
        train(model, config,dataset_train, dataset_val, augmentation)
        
        ### training resnet 
        config.TRAINABLE_LAYERS = modes[1]
        config.EPOCHS = epochs1 + epochs2  

        train(model, config,dataset_train, dataset_val, augmentation)

        #### then train everything
        config.TRAINABLE_LAYERS = modes[2]
        # train the remaining epocs
        config.EPOCHS = epochs1 + epochs2 + epochs3     
        # Train 
        train(model, config,dataset_train, dataset_val, augmentation)

    

Weights:  coco
Dataset:  /users/nfs/Etu7/21113797/PRAT/Images

Configurations:
AUGMENT_TRAIN                  True
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     6
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
EPOCHS                         20
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 6
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                2.0
IMAGE_RESIZE_MODE              crop
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0,

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import keras
print(keras.__version__)